# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Import libraries for Azure Machine Learning SDK

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as pyplot
import numpy as pd
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

# Check the core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Initialize Workspace and Experiment

In [2]:
# initialized a workspace from a persisted configuration
ws = Workspace.from_config()

project_folder = './heart_failure_proj'
# choose a name for experiment
experiment_name = 'auto-ml-heart-failure-exp'

experiment = Experiment(ws, experiment_name)

# get workspace details and print
output = {}
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group 
output['Location'] = ws.location 
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
output_df = pd.DataFrame(data = output, index=[''])
output_df.T

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


,
Subscription ID,e36180a3-997b-4765-87a8-36f70eefc1fb
Workspace,ws-airevo
Resource Group,rg-ai
Location,westus2
Experiment Name,auto-ml-heart-failure-exp


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

## Prepare the dataset for Automation

In [3]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "heart-failure-dataset"
description_text = "heart failure clinical records dataset from UCI repo "

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## Create a Compute Target

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = 'cpu-cluster'

try:
    # check for existing compute target
    training_cluster = ComputeTarget(
        workspace=ws,
        name=cluster_name,
    )
    print('Found existing cluster, use it.')
except:
    # create compute target if non exists
    try:
        compute_config = AmlCompute.provisioning_configuration(
            vm_size='STANDARD_DS12_V2',
            max_nodes=4
        )
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# define automl settings
automl_settings = {
    "n_cross_validations": 5,
    "experiment_timeout_minutes": 60,
    "max_concurrent_iterations": 2,
    "primary_metric" : 'AUC_weighted'
}

# define automl configuration settings
automl_config = AutoMLConfig(compute_target=training_cluster,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [11]:
# Submit the experiment to a remote compute cluster
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
# retrieve an existing run
from azureml.train.automl.run import AutoMLRun
remote_run = AutoMLRun(experiment = experiment, run_id = 'AutoML_c2ada1fe-bf94-48d0-802e-fcb1cc580787')

In [7]:
# show the run details of the experiment
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [8]:
# return details of the run
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|10                               |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTIO

{'runId': 'AutoML_c2ada1fe-bf94-48d0-802e-fcb1cc580787',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-09T15:34:28.28889Z',
 'endTimeUtc': '2021-01-09T16:00:54.542083Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"auto-ml-heart-failure-exp","subscription_id":"e36180a3-997b-4765-87a8-36f70eefc1fb","resource_group":"rg-ai","workspace_name":"ws-airevo","region":"westus2","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classes":null,"featurization":

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# get the best model
best_run, fitted_model = remote_run.get_output()

best_run

Experiment,Id,Type,Status,Details Page,Docs Page
auto-ml-heart-failure-exp,AutoML_c2ada1fe-bf94-48d0-802e-fcb1cc580787_32,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


'AutoML_c2ada1fe-bf94-48d0-802e-fcb1cc580787_32'

In [10]:
# display the properties of the model
fitted_model

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                random_state=None,
                                                                                                reg_alpha=0.0,
                                                                                                reg_lambda=0.0,
                                                      

In [11]:
# create inference folder
folder_name = 'inference'

if folder_name not in os.listdir():
    os.mkdir(folder_name)
    print("The folder", folder_name, "has been created")

In [12]:
# Save the best model, scoring script, and conda env files in inference folder
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
best_run.download_file('outputs/model.pkl', 'inference/model.pkl')
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'inference/conda_env.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [56]:
# Register the model
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

model_name = best_run.properties['model_name']

run_id = best_run.id
experiment_name = best_run.experiment.name

model = Model.register(workspace=ws,
                       model_name=model_name, # Name of the registered model in your workspace.
                       model_path='inference/model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,  # Version of scikit-learn used to create the model.
                       description='Auto ML model to predict motality caused by heart failure.',
                       tags={'area': 'heart-failure', 'type': 'classification'})

print('Name:', model.name)
print('Version:', model.version)

Registering model AutoMLc2ada1feb32
Name: AutoMLc2ada1feb32
Version: 4


In [60]:
# create inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment.from_conda_specification(name="myenv", file_path="inference/conda_env.yml")
inference_config = InferenceConfig(entry_script="inference/score.py", environment=myenv)

# display the environment file
with open('inference/conda_env.yml', 'r') as file:
    env_file = file.read()
    print(env_file)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-train-automl-runtime==1.19.0
  - inference-schema
  - azureml-interpret==1.19.0
  - azureml-defaults==1.19.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge



In [27]:
# Model Deployment
from azureml.core.webservice import AciWebservice

# define deployment configuration
aci_deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                           memory_gb=1,
                                                           tags={'area': "heart-failure", 'type': "classification"},
                                                           description="Predict heart failure mortality using classification model",
                                                           enable_app_insights=True)

# deploy model as webservice using Azure Container Instance(ACI)
aci_service_name = "aci-heart-failure-deploy"

aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_deployment_config, overwrite=True)
aci_service.wait_for_deployment(show_output=True)

print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


TODO: In the cell below, send a request to the web service you deployed to test it.

In [91]:
# get the active api endpoint for scoring
print(aci_service.scoring_uri)

http://04b6e591-2e1c-42a0-b80c-84bd722a908f.westus2.azurecontainer.io/score


In [105]:
import json
import requests

scoring_uri = 'http://04b6e591-2e1c-42a0-b80c-84bd722a908f.westus2.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

test_sample = json.dumps({
    "data": [
            [75.0, 0.0, 582.0, 0.0, 20.0, 1.0, 265000.0, 1.9, 130.0, 1.0, 0.0, 4.0],
            [80.0, 1.0, 123.0, 0.0, 35.0, 1.0, 388000.0, 9.4, 133.0, 1.0, 1.0, 10.0],
            [62.0, 0.0, 61.0, 1.0, 38.0, 1.0, 155000.0, 1.1, 143.0, 1.0, 1.0, 270.0],
            [50.0, 1.0, 111.0, 0.0, 20.0, 0.0, 210000.0, 1.9, 137.0, 1.0, 0.0, 7.0]
        ]
    })

response = requests.post(scoring_uri, data=test_sample, headers=headers)
print("Results:", response.json())

Results: {"result": [1, 1, 0, 1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [108]:
# print the log of the webservice
print(aci_service.get_logs())

2021-01-09T21:17:43,233978400+00:00 - rsyslog/run 
2021-01-09T21:17:43,249737600+00:00 - gunicorn/run 
2021-01-09T21:17:43,253538200+00:00 - iot-server/run 
2021-01-09T21:17:43,251260700+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [111]:
# delete the web service
aci_service.delete()
model.delete()

No service with name aci-heart-failure-deploy found to delete.
